# Git Data Extraction using Python

This notebook helps you load historical git data from any public git repository on GitHub

## Pre-requisites
The code assumes you have the following libraries installed:

- pydriller
- pandas

## Part 1: Pulling Commit Data from GitHub

We will use the PyDriller library to pull commit data from GitHub for a public repository and build a list of commits.

This process can take a very long time depending on the size of the repository.

In [1]:
# We need PyDriller to pull git repository information
from pydriller import Repository


# Using the ML.NET public repository on GitHub
account = 'dotnet'
repository = 'machinelearning'
path = 'https://github.com/' + account + '/' + repository
repo = Repository(path)

print('Using repository ' + path)

Using repository https://github.com/dotnet/machinelearning


In [2]:
def sanitize_message(msg):
    msg = msg.replace('\n', ' ')
    msg = msg.replace(',', '')
    msg = msg.replace('"', '')
    return msg

In [3]:


# Loop over each PyDriller commit to transform it to a commit usable for analysis later
# NOTE: This can take a LONG time if there are many commits

commits = []
for commit in repo.traverse_commits():

    hash = commit.hash
    try:

        # Gather a list of files modified in the commit
        files = []
        for f in commit.modified_files:
            if f.new_path is not None:
                files.append(f.new_path) 

        # Sanitize the message to prevent it from confusing our resulting CSV
        msg = sanitize_message(commit.msg)

        # Optimization to prevent requesting same data twice
        author = commit.author
        committer = commit.committer
        inserts = commit.insertions
        deletions = commit.deletions

        # Capture information about the commit in object format so I can reference it later
        record = {
            'hash': hash,
            'message': msg,
            'author_name': author.name,
            'author_email': author.email,
            'author_date': commit.author_date,
            'author_tz': commit.author_timezone,
            'committer_name': committer.name,
            'committer_email': committer.email,
            'committer_date': commit.committer_date,
            'committer_tz': commit.committer_timezone,
            'in_main': commit.in_main_branch,
            'is_merge': commit.merge,
            'num_deletes': deletions,
            'num_inserts': inserts,
            'net_lines': inserts - deletions,
            'num_files': commit.files,
            'branches': ', '.join(commit.branches), # Comma separated list of branches the commit is found in
            'files': ', '.join(files), # Comma separated list of files the commit modifies
            # PyDriller Open Source Delta Maintainability Model (OS-DMM) stat. See https://pydriller.readthedocs.io/en/latest/deltamaintainability.html for metric definitions
            'dmm_unit_size': commit.dmm_unit_size,
            'dmm_unit_complexity': commit.dmm_unit_complexity,
            'dmm_unit_interfacing': commit.dmm_unit_interfacing,
        }
        # Omitted: modified_files (list), project_path, project_name
        commits.append(record)

    except Exception as er:
        print('Problem reading commit ' + hash)
        print(er)
        continue

Problem reading commit eae76959e6714af44caa212e102a5f06f0110e72
SHA b'1ecc365249e5cac5e72c66317a141298dc52f6e3' could not be resolved, git returned: b'1ecc365249e5cac5e72c66317a141298dc52f6e3 missing'
Problem reading commit df1c2af3369a5c87ea03df17d75d7e6fa730543f
SHA b'1ecc365249e5cac5e72c66317a141298dc52f6e3' could not be resolved, git returned: b'1ecc365249e5cac5e72c66317a141298dc52f6e3 missing'
Problem reading commit c3a20faa31c22eb85806d325dfe9f12d308c772e
SHA b'f92a18161b6824fda4c4ab698a69d299a836841a' could not be resolved, git returned: b'f92a18161b6824fda4c4ab698a69d299a836841a missing'
Problem reading commit cb37c7e7f1e1b29b5608a2755db793c5435d10b1
SHA b'53a91e7e8c88463e97acfbbafb7134a6030860b3' could not be resolved, git returned: b'53a91e7e8c88463e97acfbbafb7134a6030860b3 missing'
Problem reading commit 2a927865769e10772a31af407f2a856fd6e4e523
SHA b'f92a18161b6824fda4c4ab698a69d299a836841a' could not be resolved, git returned: b'f92a18161b6824fda4c4ab698a69d299a836841a miss

## Part 2: Build a Pandas DataFrame
Now that we have a raw list of commits available, let's translate that to a Pandas DataFrame so we can validate the data appears roughly correct before exporting

In [4]:

import pandas as pd

# Translate this list of commits to a Pandas data frame, then export it to CSV for analysis
df_commits = pd.DataFrame(commits)
df_commits.head()

,hash,message,author_name,author_email,author_date,author_tz,committer_name,committer_email,committer_date,committer_tz,...,is_merge,num_deletes,num_inserts,net_lines,num_files,branches,files,dmm_unit_size,dmm_unit_complexity,dmm_unit_interfacing
0,f0e639af5ffdc839aae8e65d19b5a9a1f0db634a,Initial commit,dotnet-bot,dotnet-bot@microsoft.com,2018-05-03 17:22:00-07:00,25200,Immo Landwerth,immol@microsoft.com,2018-05-03 17:22:00-07:00,25200,...,False,0,382168,382168,868,main,".gitattributes, .gitignore, BuildToolsVersion....",0.399491,0.611602,0.630582
1,76cb2cdf5cc8b6c88ca44b8969153836e589df04,Get a working build (#1) * Set missing execut...,Sandy Armstrong,sanfordarmstrong@gmail.com,2018-05-04 12:47:21-07:00,25200,Eric Erhardt,eric.erhardt@microsoft.com,2018-05-04 14:47:21-05:00,18000,...,False,27,1749,1722,23,main,"Microsoft.ML.sln, build.sh, init-tools.sh, run...",NaN,NaN,NaN
2,972f6232de173b5e294a34a847682e9b1e67d3af,Fixed the syntax of cited example. (#2),Zeeshan Ahmed,38438266+zeahmed@users.noreply.github.com,2018-05-04 14:06:13-07:00,25200,Eric Erhardt,eric.erhardt@microsoft.com,2018-05-04 16:06:13-05:00,18000,...,False,5,4,-1,1,main,README.md,NaN,NaN,NaN
3,cde0d7d18ec9e93bde1d3a53c35f87430ac43fee,Add ML.NET Roadmap (#30) ##Add Roadmap.md for...,Gleb K,glebk@microsoft.com,2018-05-05 01:11:31-07:00,25200,GitHub,noreply@github.com,2018-05-05 01:11:31-07:00,25200,...,False,1,128,127,3,main,"Microsoft.ML.sln, README.md, ROADMAP.md",NaN,NaN,NaN
4,979418886950e144b2cc561bdc5eb41d382cf829,Update contribution guide and issue/PR templates,Shauheen Zahirazami,shzahira@microsoft.com,2018-05-05 13:47:44-07:00,25200,Shauheen Zahirazami,shzahira@microsoft.com,2018-05-05 13:47:44-07:00,25200,...,False,0,53,53,3,main,"CONTRIBUTING.md, ISSUE_TEMPLATE.md, PULL_REQUE...",NaN,NaN,NaN


In [5]:
# Look at the trends in the OS-DMM metrics
df_commits[['dmm_unit_complexity', 'dmm_unit_interfacing', 'dmm_unit_size']].describe()

,dmm_unit_complexity,dmm_unit_interfacing,dmm_unit_size
count,1486.000000,1492.000000,1485.000000
mean,0.678876,0.658474,0.400327
std,0.412150,0.403686,0.404094
min,0.000000,0.000000,0.000000
25%,0.254870,0.279778,0.000000
50%,0.974098,0.891310,0.263636
75%,1.000000,1.000000,0.857143
max,1.000000,1.000000,1.000000


## Part 3: Export to a CSV File
Because the commit extraction process takes a very long time, let's export the resulting data to a CSV file so we don't need to repeat it every time we need to analyze the data

In [6]:
df_commits.to_csv('Commits.csv')

## Part 4: Breakdown by file

In [7]:
commits = []

for commit in repo.traverse_commits():
    hash = commit.hash
    try:
        # Sanitize the message to prevent it from confusing our resulting CSV
        msg = sanitize_message(commit.msg)

        # Optimization to prevent requesting same data twice
        author = commit.author
        committer = commit.committer
        inserts = commit.insertions
        deletions = commit.deletions
        author_date = commit.author_date
        author_timezone = commit.author_timezone
        committer_date = commit.committer_date
        committer_timezone = commit.committer_timezone
        in_main_branch = commit.in_main_branch
        is_merge = commit.merge
        branches = ', '.join(commit.branches) # Comma separated list of branches the commit is found in
        project_name = commit.project_name
        project_path = commit.project_path

        for f in commit.modified_files:
            record = {
                'hash': hash,
                'message': msg,
                'author_name': author.name,
                'author_email': author.email,
                'author_date': author_date,
                'author_tz': author_timezone,
                'committer_name': committer.name,
                'committer_email': committer.email,
                'committer_date': committer_date,
                'committer_tz': committer_timezone,
                'in_main': in_main_branch,
                'is_merge': is_merge,
                'num_deletes': deletions,
                'num_inserts': inserts,
                'net_lines': inserts - deletions,
                'branches': branches,
                'filename': f.filename,
                'old_path': f.old_path,
                'new_path': f.new_path,
                'project_name': project_name,
                'project_path': project_path, 
            }
            # Omitted: modified_files (list), project_path, project_name
            commits.append(record)
    except Exception as er:
        print('Problem reading commit ' + hash)
        print(er)
        continue        

Problem reading commit eae76959e6714af44caa212e102a5f06f0110e72
SHA b'1ecc365249e5cac5e72c66317a141298dc52f6e3' could not be resolved, git returned: b'1ecc365249e5cac5e72c66317a141298dc52f6e3 missing'
Problem reading commit df1c2af3369a5c87ea03df17d75d7e6fa730543f
SHA b'1ecc365249e5cac5e72c66317a141298dc52f6e3' could not be resolved, git returned: b'1ecc365249e5cac5e72c66317a141298dc52f6e3 missing'
Problem reading commit c3a20faa31c22eb85806d325dfe9f12d308c772e
SHA b'f92a18161b6824fda4c4ab698a69d299a836841a' could not be resolved, git returned: b'f92a18161b6824fda4c4ab698a69d299a836841a missing'
Problem reading commit cb37c7e7f1e1b29b5608a2755db793c5435d10b1
SHA b'53a91e7e8c88463e97acfbbafb7134a6030860b3' could not be resolved, git returned: b'53a91e7e8c88463e97acfbbafb7134a6030860b3 missing'
Problem reading commit 2a927865769e10772a31af407f2a856fd6e4e523
SHA b'f92a18161b6824fda4c4ab698a69d299a836841a' could not be resolved, git returned: b'f92a18161b6824fda4c4ab698a69d299a836841a miss

In [8]:
import pandas as pd

# Translate this list of commits to a Pandas data frame, then export it to CSV for analysis
df_file_commits = pd.DataFrame(commits)

df_file_commits.to_csv('FileCommits.csv')

df_file_commits.head()

,hash,message,author_name,author_email,author_date,author_tz,committer_name,committer_email,committer_date,committer_tz,...,is_merge,num_deletes,num_inserts,net_lines,branches,filename,old_path,new_path,project_name,project_path
0,f0e639af5ffdc839aae8e65d19b5a9a1f0db634a,Initial commit,dotnet-bot,dotnet-bot@microsoft.com,2018-05-03 17:22:00-07:00,25200,Immo Landwerth,immol@microsoft.com,2018-05-03 17:22:00-07:00,25200,...,False,0,382168,382168,main,.gitattributes,None,.gitattributes,machinelearning,C:\Users\Admin\AppData\Local\Temp\tmpaoyft4wz\...
1,f0e639af5ffdc839aae8e65d19b5a9a1f0db634a,Initial commit,dotnet-bot,dotnet-bot@microsoft.com,2018-05-03 17:22:00-07:00,25200,Immo Landwerth,immol@microsoft.com,2018-05-03 17:22:00-07:00,25200,...,False,0,382168,382168,main,.gitignore,None,.gitignore,machinelearning,C:\Users\Admin\AppData\Local\Temp\tmpaoyft4wz\...
2,f0e639af5ffdc839aae8e65d19b5a9a1f0db634a,Initial commit,dotnet-bot,dotnet-bot@microsoft.com,2018-05-03 17:22:00-07:00,25200,Immo Landwerth,immol@microsoft.com,2018-05-03 17:22:00-07:00,25200,...,False,0,382168,382168,main,BuildToolsVersion.txt,None,BuildToolsVersion.txt,machinelearning,C:\Users\Admin\AppData\Local\Temp\tmpaoyft4wz\...
3,f0e639af5ffdc839aae8e65d19b5a9a1f0db634a,Initial commit,dotnet-bot,dotnet-bot@microsoft.com,2018-05-03 17:22:00-07:00,25200,Immo Landwerth,immol@microsoft.com,2018-05-03 17:22:00-07:00,25200,...,False,0,382168,382168,main,CONTRIBUTING.md,None,CONTRIBUTING.md,machinelearning,C:\Users\Admin\AppData\Local\Temp\tmpaoyft4wz\...
4,f0e639af5ffdc839aae8e65d19b5a9a1f0db634a,Initial commit,dotnet-bot,dotnet-bot@microsoft.com,2018-05-03 17:22:00-07:00,25200,Immo Landwerth,immol@microsoft.com,2018-05-03 17:22:00-07:00,25200,...,False,0,382168,382168,main,Directory.Build.props,None,Directory.Build.props,machinelearning,C:\Users\Admin\AppData\Local\Temp\tmpaoyft4wz\...
